In [1]:
import flyte
import flyte.git
import flyte.storage

In [2]:
flyte.init_from_config(flyte.git.config_from_root(), storage=flyte.storage.S3.auto(region="us-east-2"))

In [3]:
env = flyte.TaskEnvironment(name="pd_data", image=flyte.Image.from_debian_base().with_pip_packages("pandas", "pyarrow"))

In [4]:
import pandas as pd

BASIC_EMPLOYEE_DATA = {
    "employee_id": range(1001, 1009),
    "name": ["Alice", "Bob", "Charlie", "Diana", "Ethan", "Fiona", "George", "Hannah"],
    "department": ["HR", "Engineering", "Engineering", "Marketing", "Finance", "Finance", "HR", "Engineering"],
    "hire_date": pd.to_datetime(
        ["2018-01-15", "2019-03-22", "2020-07-10", "2017-11-01", "2021-06-05", "2018-09-13", "2022-01-07", "2020-12-30"]
    ),
}

In [5]:
@env.task
async def create_raw_dataframe() -> pd.DataFrame:
    """
    This task creates a raw pandas DataFrame with basic employee information.
    This is the most basic use-case of how to pass dataframes (of all kinds, not just pandas). Create the dataframe
    as normal, and return it. Note that the output signature is of the dataframe library type.
    Uploading of the actual bits of the dataframe (which for pandas is serialized to parquet) happens at the
    end of the task, the TypeEngine uploads from memory to blob store.
    """
    return pd.DataFrame(BASIC_EMPLOYEE_DATA)

In [6]:
r = flyte.run(create_raw_dataframe)

11:08:24.008796 WARNING  remote_builder.py:86 -  Image 356633062068.dkr.ecr.us-east-2.amazonaws.com/union/demo:flyte-e021d674b86f19634c7597c810ec9edd found.    
                         Skip building.

11:08:24.017920 WARNING  _deploy.py:160 -  Built Image for environment pd_data, image:                                                                          
                         356633062068.dkr.ecr.us-east-2.amazonaws.com/union/demo:flyte-e021d674b86f19634c7597c810ec9edd

In [7]:
r.url

'https://demo.hosted.unionai.cloud/v2/runs/project/flytesnacks/domain/development/r4l9zlzqvqqhvxs529nw'

In [8]:
r.wait()

/Users/johnvotta/code/flyte-sdk/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/johnvotta/code/flyte-sdk/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Run 'r4l9zlzqvqqhvxs529nw' completed successfully.

In [9]:
r.inputs()

{}

In [10]:
outputs = r.outputs()

In [11]:
outputs.pb2

literals {
  name: "o0"
  value {
    scalar {
      structured_dataset {
        uri: "s3://union-oc-production-demo/metadata/v2/demo/flytesnacks/development/r4l9zlzqvqqhvxs529nw/a0/1/io/r4l9zlzqvqqhvxs529nw-a0-0/e59a93079bf6ace39cebdaf56d5a98e5"
        metadata {
          structured_dataset_type {
            format: "parquet"
          }
        }
      }
    }
  }
}

In [12]:
fdf = outputs[0]

In [13]:
await fdf.open(pd.DataFrame).all()

,employee_id,name,department,hire_date
0,1001,Alice,HR,2018-01-15
1,1002,Bob,Engineering,2019-03-22
2,1003,Charlie,Engineering,2020-07-10
3,1004,Diana,Marketing,2017-11-01
4,1005,Ethan,Finance,2021-06-05
5,1006,Fiona,Finance,2018-09-13
6,1007,George,HR,2022-01-07
7,1008,Hannah,Engineering,2020-12-30
